# PyTorch Object detection with ScaledYolov4

We can use Scaled Yolov4 to run our first object detector.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.nn.functional as F
import torchvision.models as models

from PIL import Image, ImageFilter, ImageChops
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_and_extract_archive

import requests
import copy
import os
import random
from io import BytesIO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dependencies

### Install ScaledYoloV4

In [2]:
#!git clone https://github.com/roboflow-ai/ScaledYOLOv4.git
# the following fork has a few fixes which allow training with 
# newer numpy versions
!git clone https://github.com/schms27/ScaledYOLOv4.git

%cd ./ScaledYOLOv4/
!git checkout yolov4-large

fatal: destination path 'ScaledYOLOv4' already exists and is not an empty directory.
/app/notebooks/ScaledYOLOv4
Already on 'yolov4-large'
Your branch is up to date with 'origin/yolov4-large'.


### Install mish activation function

In [3]:
%cd /app/notebooks
%ls
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install

/app/notebooks
01_getting_started_opencv_image_loading.ipynb*
02_color_spaces.ipynb*
03_draw_to_image.ipynb*
04_transformations.ipynb*
05_arithmetic_ops.ipynb*
06_thresholding_conv_sharpening_blurring.ipynb.ipynb*
07_morphology.ipynb*
08_contours.ipynb*
09_hough_and_blob.ipynb*
10_simple_feature_detection_template_matching.ipynb*
11_haar_cascade_detector.ipynb*
12_object_tracking.ipynb*
13_facial_landmark_detection.ipynb*
14_video_streaming.ipynb*
30_cnn_pytorch_mnist.ipynb*
31_pytorch_regularization.ipynb*
32_pytorch_filter_visualization.ipynb*
33_pytorch_load_pretrained_cnns.ipynb*
34_pytorch_lightning.ipynb*
35_pytorch_lightning_transfer_learning.ipynb*
36_pytorch_deep_dream.ipynb*
38_gan.ipynb*
39_siamese_networks.ipynb*
40_face_recognition.ipynb
41_object_detection_scaled_yolov4.ipynb*
MNIST/
ScaledYOLOv4/
lightning_logs/
mish-cuda/
out/
tmp/
weights/
fatal: destination path 'mish-cuda' already exists and is not an empty directory.
/app/notebooks/mish-cuda
/home/user/mambaforge/li

In [4]:
!pip install -U PyYAML

  Obtaining dependency information for PyYAML from https://files.pythonhosted.org/packages/7d/39/472f2554a0f1e825bd7c5afc11c817cd7a2f3657460f7159f691fbb37c51/PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [5]:
%cd /app/notebooks/ScaledYOLOv4/

/app/notebooks/ScaledYOLOv4


## Download dataset
We use a public dataset from roboflow which contains annotated images of handguns.

https://public.roboflow.com/object-detection/pistols/1

For the following to work, you need a roboflow account and fill in your api_key:

In [6]:
!pip install roboflow

  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/ca/02/f34b5111890e235b831596cc596b219148ad8aa6b45939f2dc6d68da3404/roboflow-1.1.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.6 MB/s eta 0:00:00
  Obtaining dependency information for supervision from https://files.pythonhosted.org/packages/ca/e8/4d0e1f866ae4cad3bebdfbd91d1e257a53090e103499a3e77c461e28f1b5/supervision-0.13.0-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for opencv-python-headless<5.0.0.0,>=4.8.0.74 from https://files.pythonhosted.org/packages/76/02/f128517f3ade4bb5f71e2afd8461dba70e3f466ce745fa1fd1fade9ad1b7/opencv_python_headless-4.8.0.

In [24]:
%cd /app/notebooks/tmp

/app/notebooks/tmp


In [10]:
from roboflow import Roboflow
API_KEY = "ywc2sP73cgLeBcBqMh0p"#<YOUR_API_KEY>"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("joseph-nelson").project("pistols")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Extracting Dataset Version Zip to Pistols-1 in yolov5pytorch:: 100%|██████████| 5945/5945 [00:30<00:00, 193.85it/s]


In [36]:
%cd /app/notebooks/ScaledYOLOv4

/app/notebooks/ScaledYOLOv4


In [37]:
from roboflow import Roboflow

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("joseph-nelson").project("pistols")
dataset = project.version(1).download("yolov4scaled")

loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Extracting Dataset Version Zip to Pistols-1 in yolov4scaled:: 100%|██████████| 5947/5947 [00:26<00:00, 225.04it/s]


We have to rename some folders

In [38]:
%cd ./Pistols-1
%mv ./export ./train

/app/notebooks/ScaledYOLOv4/Pistols-1


This dataset contains a data.yaml file which describes the dataset.

Change data.yaml to look like this:

```
train: ./Pistols-1/train/images
val: ./Pistols-1/valid/images

nc: 1
names: ['pistol']
```

In [39]:
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['pistol']

## YoloV4 Model architecture
We can have a look at the ScaledYolov4 Model architecture.

In [40]:
%cat /app/notebooks/ScaledYOLOv4/models/yolov4-csp.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov4-csp backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Bottleneck, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 2, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 8, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 8, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 4, BottleneckCSP, [1024]],  # 10
  ]

# yolov4-csp head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 11
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [8, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 2, Bott

## Start training
Next, we train scaled-YOLOv4 on custom data for 100 epochs and time its performance

In [54]:
%%time
%cd /app/notebooks/ScaledYOLOv4
#%cd /app/notebooks/tmp/Pistols-1
!python train.py --img 416 --batch 16 --epochs 100 --data './Pistols-1/data.yaml' --cfg /app/notebooks/ScaledYOLOv4/models/yolov4-csp.yaml --weights '' --name yolov4-csp-results



/app/notebooks/ScaledYOLOv4
Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24575MB)

Namespace(weights='', cfg='/app/notebooks/ScaledYOLOv4/models/yolov4-csp.yaml', data='./Pistols-1/data.yaml', hyp='data/hyp.scratch.yaml', epochs=100, batch_size=16, img_size=[416, 416], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, name='yolov4-csp-results', device='', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, logdir='runs/', total_batch_size=16, world_size=1, global_rank=-1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.5, 'scale': 0.5, 'shear': 0.0, 'perspe